In [0]:
!sudo apt update  
!sudo apt install apt-transport-https ca-certificates curl software-properties-common
!curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -
!sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"
!sudo apt update
!sudo apt install docker-ce


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done
Building dependency tree       
Reading state information... Done
41 packages can be upgraded. Run 'apt

In [0]:
!sudo systemctl status docker


System has not been booted with systemd as init system (PID 1). Can't operate.


In [0]:
# Load training data to colab space

# mount local drive: works with CPU, not with GPU
#from google.colab import drive
#drive.mount('/content/drive/')

# load file from specific link, didn't work for me
#download=drive.CreateFile({'id':'https://drive.google.com/open?id=1-s9pfnlG8Ln7eJRfJhr1PlX17Kzou5jW'})
#download.GetContentFile('training3.zip')
#!unzip training3.zip

# generic link to local drive, need to specify file through GUI 
#from google.colab import files
#files.upload()
#import os
#os.rename('../training5.zip', 'training5.zip')
!unzip training5.zip


# or just upload from Files menu to left

In [0]:
!pip install -q keras
import keras
print(keras.__version__)

Using TensorFlow backend.


2.2.4


In [0]:
# train only top classification layer, base model frozen

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import MobileNetV2
from keras.applications.mobilenet import preprocess_input
from keras import backend as K
import numpy as np
import os


# config parameters
image_size = 160 # All images will be resized to 160x160
batch_size = 32
base_dir='training5'

IMG_SHAPE = (image_size, image_size, 3)
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                rotation_range=10,
                width_shift_range=.05,
                height_shift_range=0.05)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),  
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

# Flow validation images in batches using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')


# get pre-trained model
base_model=MobileNetV2(weights='imagenet',
                       include_top=False,
                       input_shape=IMG_SHAPE)
base_model.trainable=False

# add layers to model
model = keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1, activation='sigmoid')
])


# compile model
#model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001), 
#model.compile(optimizer=keras.optimizers.RMSprop(lr=0.001), 
model.compile(optimizer='Adam',               
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# train model
epochs = 10
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

history = model.fit_generator(train_generator, 
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs, 
                              workers=4,
                              #use_multiprocessing=True ,
                              validation_data=validation_generator, 
                              validation_steps=validation_steps)


#save results
model.save('sw_street_v2_15epoch_p001.h5')

Found 21414 images belonging to 2 classes.
Found 3504 images belonging to 2 classes.


W0628 21:24:09.369825 140033442740096 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0628 21:24:09.410164 140033442740096 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0628 21:24:09.422384 140033442740096 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0628 21:24:09.455877 140033442740096 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0628 21:24:09.457960 1400334427

9412608/9406464 [==============================] - 1s 0us/step


W0628 21:24:25.367329 140033442740096 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0628 21:24:25.394745 140033442740096 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
669/669 [==============================] - 113s 168ms/step - loss: 0.3319 - acc: 0.8576 - val_loss: 0.3846 - val_acc: 0.8320
Epoch 2/10
669/669 [==============================] - 103s 154ms/step - loss: 0.2562 - acc: 0.8934 - val_loss: 0.3337 - val_acc: 0.8618
Epoch 3/10
362/669 [===============>..............] - ETA: 44s - loss: 0.2368 - acc: 0.9046

KeyboardInterrupt: ignored

In [0]:
#save results
model.save('coral_test.h5')

In [0]:
# plot training history

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.show()

In [0]:
# Modify model to make top 55 (of 155) layers trainable

base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 55

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False
  
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.00002),
              metrics=['accuracy'])

model.summary()

In [0]:
# do fine tuning of top layers specified above
epochs=5
history_fine = model.fit_generator(train_generator, 
                                   steps_per_epoch = steps_per_epoch,
                                   epochs=epochs, 
                                   workers=4,
                                   validation_data=validation_generator, 
                                   validation_steps=validation_steps)

In [0]:
# look at prediction errors

import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img


# Get the filenames from the generator
fnames = validation_generator.filenames
 
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(validation_generator,
                                      steps=validation_generator.samples/validation_generator.batch_size,
                                      verbose=1)
predictions=np.squeeze(predictions)
#predicted_classes = np.argmax(predictions,axis=1)
predicted_classes=predictions.astype(int)
 
errors = np.where(predicted_classes != ground_truth)
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))
 
# Show the errors
for i in range(100):
    #pred_class = np.argmax(predictions[errors[i]])
    pred_class=predicted_classes[errors[i]]
    pred_label = idx2label[pred_class]
     
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][0])
     
    original = load_img('{}/{}'.format(validation_dir,fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()

In [0]:
#save results
model.save('mobileV2_5epochs_100layer_0516.h5')

In [0]:


import numpy as np
import cv2
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing import image
from keras.models import load_model
import os
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img

dirIn = 'training5/train/street'
modelName='mobileV2_20epochs_0516.h5'
loadModel=0


if loadModel:
    model=load_model(modelName)
else:
    print('Model not loaded')
    



# loop through all images in the folder
preds=[]
fnum=0
for file in os.listdir(dirIn):
    if file[-3:]!='jpg':
      continue
    frame= cv2.imread(os.path.join(dirIn,file))
    fnum+=1
    #print(fnum)
   
    # pre-process image
    img = cv2.resize(frame, (160,160), interpolation = cv2.INTER_AREA)
      
    # convert to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img=np.divide(img,255)

    x = np.expand_dims(img, axis=0)

    #x = preprocess_input(x)
    
    # classify image
    preds.append(model.predict(x)[0][0])
    if preds[-1]<0.5:
      original = load_img('{}/{}'.format(dirIn,file))
      plt.figure(figsize=[7,7])
      plt.axis('off')
      plt.title(file)
      plt.imshow(original)
      plt.show()
      

  

p=np.asarray(preds)    
errors=np.where(p>0.5)








In [0]:
import os
import tensorflow as tf
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    base_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))
